# Notebook to Extract Drug-Target Interactions Based on ChEMBL Data

### Authors: Barbara Zdrazil, Lina Heinzke
### 12/2022

This notebook extracts data from ChEMBL in order to retrieve a data set for drug-target and clinical candidate-target associations as well as comparator compounds for the respective targets.

The notebook is based on initial work by Anne Hersey, Patrica Bento, Emma Manners, Paul Leeson, and Andrew Leach:  
*Target-Based Evaluation of “Drug-Like” Properties and Ligand Efficiencies  
Paul D. Leeson, A. Patricia Bento, Anna Gaulton, Anne Hersey, Emma J. Manners, Chris J. Radoux, and Andrew R. Leach  
J. Med. Chem. 2021, 64, 11, 7210–7230  
[DOI: 10.1021/acs.jmedchem.1c00416](https://doi.org/10.1021/acs.jmedchem.1c00416)*


More documentation on the initial data set compilation can be found here ("Ligand Efficiency"): https://www.ebi.ac.uk/seqdb/confluence/pages/viewpage.action?spaceKey=CHEMBL&title=Anne%27s+Notes


In [1]:
import pandas as pd
import sqlite3
import numpy as np
from tqdm import tqdm
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem import PandasTools
from rdkit.Chem.Scaffolds import MurckoScaffold

In [2]:
# notebook settings
pd.options.display.max_rows = 100
pd.options.display.max_columns = 50
pd.options.display.max_colwidth = 100

# Get Data From ChEMBL

In [3]:
chembl_version = '32'
base_path = '/Users/heinzke/Documents/PhD/Projects/drug_target_dataset_curation/'
path_results = base_path+'results/'
path_sqlite3_database = base_path+'data/chembl_'+chembl_version+'/chembl_'+chembl_version+'_sqlite/chembl_'+chembl_version+'.db'
chembl_con = sqlite3.connect(path_sqlite3_database)

Initial query for activities + related assay, mutation, target und docs information.

In [4]:
sql = '''
SELECT act.pchembl_value, 
    md.molregno as parent_molregno, md.chembl_id as parent_chemblid, md.pref_name as parent_pref_name,
    md.max_phase, md.first_approval, md.usan_year, md.black_box_warning, 
    md.prodrug, md.oral, md.parenteral, md.topical, 
    ass.assay_type, ass.tid, 
    vs.mutation,
    td.chembl_id as target_chembl_id, td.pref_name as target_pref_name, td.target_type, td.organism, 
    docs.year
FROM activities act
INNER JOIN molecule_hierarchy mh 
    ON act.molregno = mh.molregno         -- act.molregno = salt_molregno
INNER JOIN molecule_dictionary md
    ON mh.parent_molregno = md.molregno   -- compound information based on parent compound
INNER JOIN assays ass 
    ON  act.assay_id = ass.assay_id
LEFT JOIN variant_sequences vs
    ON ass.variant_id = vs.variant_id
INNER JOIN target_dictionary td
    ON ass.tid = td.tid
LEFT JOIN docs
    ON act.doc_id = docs.doc_id
WHERE act.pchembl_value is not null
    and act.potential_duplicate = 0
    and act.standard_relation = '='
    and data_validity_comment is null
    and td.tid <>22226                    -- exclude unchecked targets
    and td.target_type like '%PROTEIN%'
'''

df_mols = pd.read_sql_query(sql, con=chembl_con)
# target_id_mutation
df_mols['tid_mutation'] = np.where(df_mols['mutation'].notnull(), 
                                   df_mols['tid'].astype('str')+'_'+df_mols['mutation'], 
                                   df_mols['tid'].astype('str'))
# compound-target association
df_mols['cpd_target_pair'] = df_mols.agg('{0[parent_molregno]}_{0[tid]}'.format, axis=1)
df_mols

,pchembl_value,parent_molregno,parent_chemblid,parent_pref_name,max_phase,first_approval,usan_year,black_box_warning,prodrug,oral,parenteral,topical,assay_type,tid,mutation,target_chembl_id,target_pref_name,target_type,organism,year,tid_mutation,cpd_target_pair
0,5.40,252199,CHEMBL357278,None,NaN,NaN,NaN,0,-1,0,0,0,B,10483,None,CHEMBL4632,Palmitoyl-CoA oxidase,SINGLE PROTEIN,Rattus norvegicus,2004.0,10483,252199_10483
1,4.77,253534,CHEMBL357119,None,NaN,NaN,NaN,0,-1,0,0,0,B,10483,None,CHEMBL4632,Palmitoyl-CoA oxidase,SINGLE PROTEIN,Rattus norvegicus,2004.0,10483,253534_10483
2,6.75,253199,CHEMBL152968,None,NaN,NaN,NaN,0,-1,0,0,0,B,10483,None,CHEMBL4632,Palmitoyl-CoA oxidase,SINGLE PROTEIN,Rattus norvegicus,2004.0,10483,253199_10483
3,5.22,253199,CHEMBL152968,None,NaN,NaN,NaN,0,-1,0,0,0,A,12594,None,CHEMBL3356,Cytochrome P450 1A2,SINGLE PROTEIN,Homo sapiens,2004.0,12594,253199_12594
4,4.43,253199,CHEMBL152968,None,NaN,NaN,NaN,0,-1,0,0,0,A,17045,None,CHEMBL340,Cytochrome P450 3A4,SINGLE PROTEIN,Homo sapiens,2004.0,17045,253199_17045
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2692677,6.84,2730009,CHEMBL5085558,None,NaN,NaN,NaN,0,-1,0,0,0,B,101496,None,CHEMBL5668,Serine/threonine-protein kinase SRPK2,SINGLE PROTEIN,Homo sapiens,NaN,101496,2730009_101496
2692678,7.46,2517208,CHEMBL4780617,None,NaN,NaN,NaN,0,-1,0,0,0,B,105691,None,CHEMBL2362979,Probable global transcription activator SNF2L2,SINGLE PROTEIN,Homo sapiens,NaN,105691,2517208_105691
2692679,7.44,2517208,CHEMBL4780617,None,NaN,NaN,NaN,0,-1,0,0,0,B,107968,None,CHEMBL3085620,Transcription activator BRG1,SINGLE PROTEIN,Homo sapiens,NaN,107968,2517208_107968
2692680,7.89,2517208,CHEMBL4780617,None,NaN,NaN,NaN,0,-1,0,0,0,B,104264,None,CHEMBL1795184,Protein polybromo-1,SINGLE PROTEIN,Homo sapiens,NaN,104264,2517208_104264


Set correct types.

In [5]:
df_mols = df_mols.astype({
    'year': 'Int64',
    'usan_year': 'Int64',
    'first_approval': 'Int64'
})

In [6]:
# df_mols.to_csv(path_results+"ChEMBL"+chembl_version+"_initial_query.csv", sep = ';', index = False)

In [7]:
############### TESTING: method to save dataset size at any given point to array ###############
# assay with sizes of full dataset
all_lengths = []
# assay with sizes of dataset with pchembl values
all_lengths_pchembl = []

def calculate_dataset_sizes(data):
    now_mols = len(set(data["parent_molregno"]))
    now_targets = len(set(data["tid"]))
    now_targets_mutation = len(set(data["tid_mutation"]))
    now_pairs = len(set(data['cpd_target_pair']))
    now_pairs_mutation = len(set(data['parent_molregno_tid_mutation']))
    
    if 'DTI' in data.columns:
        data_drugs = data[data["DTI"] == "D_DT"]
    else: 
        data_drugs = data[data["max_phase"] == 4]
        
    now_drugs = len(set(data_drugs["parent_molregno"]))
    now_drug_targets = len(set(data_drugs["tid"]))
    now_drug_targets_mutation = len(set(data_drugs["tid_mutation"]))
    now_drug_pairs = len(set(data_drugs['cpd_target_pair']))
    now_drug_pairs_mutation = len(set(data_drugs['parent_molregno_tid_mutation']))
    

    return [now_mols, now_drugs, 
            now_targets, now_drug_targets,
            now_targets_mutation, now_drug_targets_mutation,
            now_pairs, now_drug_pairs,
            now_pairs_mutation, now_drug_pairs_mutation]

def add_dataset_sizes(data, label, output=False):
    data_test = data.copy()
    data_test['parent_molregno_tid_mutation'] = data_test.agg('{0[parent_molregno]}_{0[tid_mutation]}'.format, axis=1)
    
    all_lengths.append([label] + calculate_dataset_sizes(data_test))
    
    # restrict to data with pchembl value (any data, even if it is only functional data)
    data_pchembl = data_test.dropna(subset=[x for x in data_test.columns if x.startswith('pchembl_value')], how='all')
    all_lengths_pchembl.append([label] + calculate_dataset_sizes(data_pchembl))

In [8]:
############### TESTING: initial query ###############
add_dataset_sizes(df_mols, "init", True)

# Calculate Mean, Median, and Max *pchembl* Values for Each Compound-Target Pair

The following values are set to summarise the information for compound-target pairs:  

|||
| :----------- | :----------- |
| *pchembl_value_mean* | mean pchembl value for a compound-target pair|
| *pchembl_value_max*| maximum pchembl value for a compound-target pair|
| *pchembl_value_median*| median pchembl value for a compound-target pair|
| *first_publication_cpd_target_pair* | first publication in ChEMBL with this compound-target pair |
| *first_publication_cpd_target_pair_w_pchembl* | first publication in ChEMBL with this compound-target pair and an associated pchembl value |

The values are set for a table with binding and functional assay data and another table with only binding assay data. These tables are combined into one table for further handling and can be distinguished by the parameter only_binding (binding and functional assay data = False; only binding data = True).

In [9]:
def get_average_info(df, assay_desc):
    df['pchembl_value_mean_'+assay_desc] = df.groupby(['parent_molregno', 'tid_mutation'])['pchembl_value'].transform('mean')
    df['pchembl_value_max_'+assay_desc] = df.groupby(['parent_molregno', 'tid_mutation'])['pchembl_value'].transform('max')
    df['pchembl_value_median_'+assay_desc] = df.groupby(['parent_molregno', 'tid_mutation'])['pchembl_value'].transform('median')
    df['first_publication_cpd_target_pair_'+assay_desc] = df.groupby(['parent_molregno', 'tid_mutation'])['year'].transform('min')
    df_mols_all_first_publication_pchembl = df[~df['pchembl_value'].isnull()] \
            .groupby(['parent_molregno', 'tid_mutation'])['year'].min().reset_index() \
            .rename(columns={'year': 'first_publication_cpd_target_pair_w_pchembl_'+assay_desc})
    df = df.merge(df_mols_all_first_publication_pchembl, on=['parent_molregno', 'tid_mutation'], how='left')
    df = df.drop(columns=['pchembl_value', 'year', 'assay_type']).drop_duplicates()
    df = df[['parent_molregno', 'tid_mutation', 
            'pchembl_value_mean_'+assay_desc, 'pchembl_value_max_'+assay_desc, 'pchembl_value_median_'+assay_desc, 
            'first_publication_cpd_target_pair_'+assay_desc, 'first_publication_cpd_target_pair_w_pchembl_'+assay_desc]]
    return df

In [10]:
# summarise the information for binding and functional assays
df_mols_all = df_mols[(df_mols['assay_type'] == 'B') | (df_mols['assay_type'] == 'F')].copy()
assay_desc = 'BF'
df_mols_all = get_average_info(df_mols_all, assay_desc)

In [11]:
# summarise the information for only binding assays
df_mols_binding = df_mols[df_mols['assay_type'] == 'B'].copy()
assay_desc = 'B'
df_mols_binding = get_average_info(df_mols_binding, assay_desc)

In [12]:
df_combined = df_mols_all.merge(df_mols_binding, on=['parent_molregno', 'tid_mutation'], how='left')
df_combined = df_combined.merge(df_mols.drop(columns=['pchembl_value', 'year', 'assay_type']).drop_duplicates(), on=['parent_molregno', 'tid_mutation'], how='left')
df_combined

,parent_molregno,tid_mutation,pchembl_value_mean_BF,pchembl_value_max_BF,pchembl_value_median_BF,first_publication_cpd_target_pair_BF,first_publication_cpd_target_pair_w_pchembl_BF,pchembl_value_mean_B,pchembl_value_max_B,pchembl_value_median_B,first_publication_cpd_target_pair_B,first_publication_cpd_target_pair_w_pchembl_B,parent_chemblid,parent_pref_name,max_phase,first_approval,usan_year,black_box_warning,prodrug,oral,parenteral,topical,tid,mutation,target_chembl_id,target_pref_name,target_type,organism,cpd_target_pair
0,252199,10483,5.40,5.40,5.40,2004,2004,5.40,5.40,5.40,2004,2004,CHEMBL357278,None,NaN,<NA>,<NA>,0,-1,0,0,0,10483,None,CHEMBL4632,Palmitoyl-CoA oxidase,SINGLE PROTEIN,Rattus norvegicus,252199_10483
1,253534,10483,4.77,4.77,4.77,2004,2004,4.77,4.77,4.77,2004,2004,CHEMBL357119,None,NaN,<NA>,<NA>,0,-1,0,0,0,10483,None,CHEMBL4632,Palmitoyl-CoA oxidase,SINGLE PROTEIN,Rattus norvegicus,253534_10483
2,253199,10483,6.75,6.75,6.75,2004,2004,6.75,6.75,6.75,2004,2004,CHEMBL152968,None,NaN,<NA>,<NA>,0,-1,0,0,0,10483,None,CHEMBL4632,Palmitoyl-CoA oxidase,SINGLE PROTEIN,Rattus norvegicus,253199_10483
3,933,10989,8.70,8.70,8.70,2004,2004,8.70,8.70,8.70,2004,2004,CHEMBL268439,None,NaN,<NA>,<NA>,0,-1,0,0,0,10989,None,CHEMBL2186,Carbonic anhydrase XIII,SINGLE PROTEIN,Mus musculus,933_10989
4,82960,11643,4.72,4.72,4.72,1984,1984,4.72,4.72,4.72,1984,1984,CHEMBL54530,None,NaN,<NA>,<NA>,0,-1,0,0,0,11643,None,CHEMBL4320,DNA topoisomerase III,SINGLE PROTEIN,Bacillus subtilis (strain 168),82960_11643
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2237998,2728877,30006,7.24,7.24,7.24,2022,2022,7.24,7.24,7.24,2022,2022,CHEMBL5084426,None,NaN,<NA>,<NA>,0,-1,0,0,0,30006,None,CHEMBL3981,Serine/threonine-protein kinase 10,SINGLE PROTEIN,Homo sapiens,2728877_30006
2237999,2728877,100917,5.64,5.64,5.64,2022,2022,5.64,5.64,5.64,2022,2022,CHEMBL5084426,None,NaN,<NA>,<NA>,0,-1,0,0,0,100917,None,CHEMBL5319,Epithelial discoidin domain-containing receptor 1,SINGLE PROTEIN,Homo sapiens,2728877_100917
2238000,2728877,101041,6.07,6.07,6.07,2022,2022,6.07,6.07,6.07,2022,2022,CHEMBL5084426,None,NaN,<NA>,<NA>,0,-1,0,0,0,101041,None,CHEMBL5615,Serine/threonine-protein kinase MRCK gamma,SINGLE PROTEIN,Homo sapiens,2728877_101041
2238001,2499590,118046,7.08,7.08,7.08,<NA>,<NA>,7.08,7.08,7.08,<NA>,<NA>,CHEMBL4752253,None,NaN,<NA>,<NA>,0,-1,0,0,0,118046,None,CHEMBL4295801,Protein ENL,SINGLE PROTEIN,Homo sapiens,2499590_118046


# Extract Drug-Target Interactions With Disease Relevance From the drug_mechanism Table

Extract the known drug-target interactions from ChEMBL (these include some interactions between compounds with a max_phase < 4 and targets). These will be used to determine if drug-target pairs from the activities query above are known drug-target interactions. 

Note: Compound-target pairs can be in the drug_mechanisms table even though the compound is not a drug (max_phase < 4). For ease of writing, these will be referred to as drug-target interactions as well rather than compound-target pairs with a known disease-relevant interaction. 

Only entries with a disease_efficacy of 1 are taken into account, i.e., the target is believed to play a role in the efficacy of the drug.  
disease_efficacy: Flag to show whether the target assigned is believed to play a role in the efficacy of the drug in the indication(s) for which it is approved (1 = yes, 0 = no).

In [13]:
sql = '''
SELECT DISTINCT mh.parent_molregno, dm.tid
FROM drug_mechanism dm
INNER JOIN molecule_hierarchy mh
    ON dm.molregno = mh.molregno
INNER JOIN molecule_dictionary md
    ON mh.parent_molregno = md.molregno
WHERE dm.disease_efficacy = 1
    and dm.tid is not null
'''

df_dti = pd.read_sql_query(sql, con=chembl_con)
df_dti

,parent_molregno,tid
0,1124,11060
1,675068,10193
2,1125,10193
3,1085,10193
4,1124,10193
...,...,...
6258,1407411,112
6259,51961,120553
6260,51961,120554
6261,442342,22228


Query target_relations for related target ids to increase the number of target ids for which there is data in the drug_mechanisms table.
The following mappings are considered:

||||
|:------|:-----:|-----|
|protein family |-[superset of]->| single protein|
|protein complex |-[superset of]->| single protein|
|protein complex group |-[superset of]->| single protein|
|single protein |-[equivalent to]->| single protein|
|chimeric protein |-[superset of]->| single protein|
|protein-protein interaction |-[superset of]->| single protein|

For example, for *protein family -[superset of]-> single protein* this means:  
If there is a known relevant interaction between a compound and a protein family, interactions between the compound and single proteins of that protein family are considered to be known interactions as well.

In [14]:
sql = '''
SELECT tr.tid, tr.relationship, tr.related_tid, 
    td1.pref_name as pref_name_1, td1.target_type as target_type_1, td1.organism as organism_1, 
    td2.pref_name as pref_name_2, td2.target_type as target_type_2, td2.organism as organism_2 
FROM target_relations tr
INNER JOIN target_dictionary td1
    ON tr.tid = td1.tid
INNER JOIN target_dictionary td2
    ON tr.related_tid = td2.tid
'''

df_related_targets = pd.read_sql_query(sql, con=chembl_con)
df_related_targets.head()

,tid,relationship,related_tid,pref_name_1,target_type_1,organism_1,pref_name_2,target_type_2,organism_2
0,11699,SUBSET OF,104812,PI4-kinase type II,SINGLE PROTEIN,Homo sapiens,"Phosphatidylinositol 4-kinase, PI4K",PROTEIN FAMILY,Homo sapiens
1,12261,SUBSET OF,104822,c-Jun N-terminal kinase 1,SINGLE PROTEIN,Homo sapiens,"c-Jun N-terminal kinase, JNK",PROTEIN FAMILY,Homo sapiens
2,12261,SUBSET OF,118329,c-Jun N-terminal kinase 1,SINGLE PROTEIN,Homo sapiens,Mitogen-activated protein kinase 8/9,PROTEIN FAMILY,Homo sapiens
3,12755,SUBSET OF,104684,Dopamine D5 receptor,SINGLE PROTEIN,Rattus norvegicus,Dopamine receptor,PROTEIN FAMILY,Rattus norvegicus
4,12735,SUBSET OF,105018,Phosphorylase kinase gamma subunit 2,SINGLE PROTEIN,Homo sapiens,Phosphorylase kinase,PROTEIN COMPLEX GROUP,Homo sapiens


In [15]:
protein_family_mapping = df_related_targets[(df_related_targets["target_type_1"] == "PROTEIN FAMILY") 
                    & (df_related_targets["target_type_2"] == "SINGLE PROTEIN")
                    & (df_related_targets["relationship"] == "SUPERSET OF")]

protein_complex_mapping = df_related_targets[(df_related_targets["target_type_1"] == "PROTEIN COMPLEX") 
                    & (df_related_targets["target_type_2"] == "SINGLE PROTEIN")
                    & (df_related_targets["relationship"] == "SUPERSET OF")]

protein_complex_group_mapping = df_related_targets[(df_related_targets["target_type_1"] == "PROTEIN COMPLEX GROUP") 
                    & (df_related_targets["target_type_2"] == "SINGLE PROTEIN")
                    & (df_related_targets["relationship"] == "SUPERSET OF")]

single_protein_mapping = df_related_targets[(df_related_targets["target_type_1"] == "SINGLE PROTEIN") 
                    & (df_related_targets["target_type_2"] == "SINGLE PROTEIN")
                    & (df_related_targets["relationship"] == "EQUIVALENT TO")]

chimeric_protein_mapping = df_related_targets[(df_related_targets["target_type_1"] == "CHIMERIC PROTEIN") 
                    & (df_related_targets["target_type_2"] == "SINGLE PROTEIN")
                    & (df_related_targets["relationship"] == "SUPERSET OF")]

ppi_mapping = df_related_targets[(df_related_targets["target_type_1"] == "PROTEIN-PROTEIN INTERACTION") 
                    & (df_related_targets["target_type_2"] == "SINGLE PROTEIN")
                    & (df_related_targets["relationship"] == "SUPERSET OF")]

relevant_mappings = pd.concat([protein_family_mapping, 
                               protein_complex_mapping, 
                               protein_complex_group_mapping,
                               single_protein_mapping, 
                               chimeric_protein_mapping, 
                               ppi_mapping])
relevant_mappings.head()

,tid,relationship,related_tid,pref_name_1,target_type_1,organism_1,pref_name_2,target_type_2,organism_2
269,104829,SUPERSET OF,11111,Cyclooxygenase,PROTEIN FAMILY,Bos taurus,Cyclooxygenase-2,SINGLE PROTEIN,Bos taurus
270,104829,SUPERSET OF,17019,Cyclooxygenase,PROTEIN FAMILY,Bos taurus,Cyclooxygenase-1,SINGLE PROTEIN,Bos taurus
273,104745,SUPERSET OF,10868,Leukotriene B4 receptor,PROTEIN FAMILY,Homo sapiens,Leukotriene B4 receptor 2,SINGLE PROTEIN,Homo sapiens
274,104745,SUPERSET OF,10542,Leukotriene B4 receptor,PROTEIN FAMILY,Homo sapiens,Leukotriene B4 receptor 1,SINGLE PROTEIN,Homo sapiens
288,104699,SUPERSET OF,12854,Adenosine A2 receptor,PROTEIN FAMILY,Rattus norvegicus,Adenosine A2b receptor,SINGLE PROTEIN,Rattus norvegicus


Combine the drug-target-interactions (DTI) and target ids (dti_tids) from the drug mechanism table with the information based on the mapped target ids.

In [16]:
# drug-target-interactions (DTI) and target ids (dti_tids) based on the drug_mechanisms table
DTIs_original = set(df_dti.agg('{0[parent_molregno]}_{0[tid]}'.format, axis=1))
dti_tids_original = set(df_dti['tid'])

# drug-target-interactions (DTI) and target ids (dti_tids) based on mapped target ids
df_dti_add_targets = df_dti.merge(relevant_mappings, on = 'tid', how = 'inner')
DTIs_mapped = set(df_dti_add_targets.agg('{0[parent_molregno]}_{0[related_tid]}'.format, axis=1))
dti_tids_mapped = set(df_dti_add_targets['related_tid'].astype("int"))

# combined drug-target-interactions (DTI) and target ids (dti_tids) 
# based on drug_mechanisms table and mapped target ids
DTIs_set = DTIs_original.union(DTIs_mapped)
dti_tids_set = dti_tids_original.union(dti_tids_mapped)

# Add Compounds From the drug_mechanisms Table to the Dataset

Add compound target pairs from the drug_mechanism table that are not in the dataset based on the initial ChEMBL query.
These are compound-target pairs for which there is no associated pchembl value data.
Since the pairs are known interactions, they are added to the dataset despite not having a pchembl value.

In [17]:
############### TESTING: before adding additional compounds ###############
add_dataset_sizes(df_combined, "pre dm table")

Collect compound target pairs including mapped target from the drug_mechanisms table.

In [18]:
cpd_target_pairs = pd.concat([df_dti[['parent_molregno', 'tid']], 
                              df_dti_add_targets[['parent_molregno', 'related_tid']]
                              .rename(columns={'related_tid': 'tid'})]).drop_duplicates()
# None of the targets from the drug mechanism table have any mutation annotation, hence tid_mutation = tid
cpd_target_pairs['tid_mutation'] = cpd_target_pairs['tid'].astype('str')
cpd_target_pairs['cpd_target_pair'] = cpd_target_pairs.agg('{0[parent_molregno]}_{0[tid]}'.format, axis=1)

Limit the pairs to the ones that are not yet in the dataset.

In [19]:
df_combined['keep_for_binding'] = False
df_combined['keep_for_functional_binding'] = False
df_combined.loc[((~df_combined['pchembl_value_mean_B'].isnull())), 'keep_for_binding'] = True
df_combined.loc[((~df_combined['pchembl_value_mean_BF'].isnull())), 'keep_for_functional_binding'] = True

In [20]:
df_existing = df_combined.copy()
df_existing['cpd_target_pair_mutation'] = df_existing.agg('{0[parent_molregno]}_{0[tid_mutation]}'.format, axis=1)

existing_pairs_BF = set(df_existing[df_existing['keep_for_functional_binding']==True]['cpd_target_pair_mutation'])
existing_pairs_B = set(df_existing[df_existing['keep_for_binding']==True]['cpd_target_pair_mutation'])

cpd_target_pairs_B_only = cpd_target_pairs[(cpd_target_pairs['cpd_target_pair'].isin(existing_pairs_BF)) & 
                                         ~(cpd_target_pairs['cpd_target_pair'].isin(existing_pairs_B))].copy()
cpd_target_pairs_B_only['keep_for_functional_binding'] = False
cpd_target_pairs_B_only['keep_for_binding'] = True
print("#added pairs not yet present based on binding / functional assay:", len(cpd_target_pairs_B_only))

cpd_target_pairs_both = cpd_target_pairs[~(cpd_target_pairs['cpd_target_pair'].isin(existing_pairs_BF)) & 
                                         ~(cpd_target_pairs['cpd_target_pair'].isin(existing_pairs_B))].copy()
cpd_target_pairs_both['keep_for_functional_binding'] = True
cpd_target_pairs_both['keep_for_binding'] = True
print("#added pairs not yet present based on binding / functional assay:", len(cpd_target_pairs_both))


cpd_target_pairs = pd.concat([cpd_target_pairs_B_only, cpd_target_pairs_both])

#added pairs not yet present based on binding / functional assay: 139
#added pairs not yet present based on binding / functional assay: 8597


Query compound and target information and combine it with the new compound-target pairs table.

In [21]:
sql = '''
SELECT md.molregno as parent_molregno, 
    md.chembl_id as parent_chemblid, md.pref_name as parent_pref_name,
    md.max_phase, md.first_approval, md.usan_year, md.black_box_warning, 
    md.prodrug, md.oral, md.parenteral, md.topical
FROM molecule_dictionary md
'''

df_compound_info = pd.read_sql_query(sql, con=chembl_con)
cpd_target_pairs = cpd_target_pairs.merge(df_compound_info, on = 'parent_molregno', how = 'left')

In [22]:
sql = '''
SELECT td.tid, td.chembl_id as target_chembl_id, td.pref_name as target_pref_name, td.target_type, td.organism
FROM target_dictionary td
'''

df_target_info = pd.read_sql_query(sql, con=chembl_con)
cpd_target_pairs = cpd_target_pairs.merge(df_target_info, on = 'tid', how = 'left')

Combined data of existing query with new compound-target pairs.

In [23]:
df_combined = pd.concat([df_combined, cpd_target_pairs]) 

# DTI (Drug-Target Interaction) Annotations

Every compound-target pair is assigned a DTI (drug target interaction) annotation.  

The assignement is based on three questions:
- Is the compound-target pair in the drug_mechanisms table? = Is it a known relevant compound-target interaction?
- What is the max_phase of the compound? = Is it a drug / clinical compound?
- Is the target in the drug_mechanisms table = Is it a therapeutic target?

The assigments are based on the following table:

|in drug_mechanisms table?|max_phase?|therapeutic target?|DTI annotation|explanation|
|:-----:|:-----:|:-----:|:-----:|:-----|
|yes|4|-|D_DT|drug - drug target|
|yes|3|-|C3_DT|clinical candidate in phase 3 - drug target|
|yes|2|-|C2_DT|clinical candidate in phase 2 - drug target|
|yes|1|-|C1_DT|clinical candidate in phase 1 - drug target|
|yes|0|-|C0_DT|compound in phase 0 - drug target|
|no|-|yes|DT|drug target|
|no|-|no|NDT|not drug target|




Identify which targets are therapeutic targets (= are they in the drug_mechanism table?) and add the field *therapeutic_target* that indicates whether target is a known therapeutic target.  

In [24]:
df_combined['therapeutic_target'] = df_combined['tid'].isin(dti_tids_set)

Assign the annotations based on the table.

In [25]:
df_combined.loc[(df_combined['cpd_target_pair'].isin(DTIs_set) & (df_combined['max_phase'] == 4)), 'DTI'] = "D_DT"
df_combined.loc[(df_combined['cpd_target_pair'].isin(DTIs_set) & (df_combined['max_phase'] == 3)), 'DTI'] = "C3_DT"
df_combined.loc[(df_combined['cpd_target_pair'].isin(DTIs_set) & (df_combined['max_phase'] == 2)), 'DTI'] = "C2_DT"
df_combined.loc[(df_combined['cpd_target_pair'].isin(DTIs_set) & (df_combined['max_phase'] == 1)), 'DTI'] = "C1_DT"
df_combined.loc[(df_combined['cpd_target_pair'].isin(DTIs_set) & (df_combined['max_phase'] == 0)), 'DTI'] = "C0_DT"
df_combined.loc[((~df_combined['cpd_target_pair'].isin(DTIs_set)) 
                 & (df_combined['therapeutic_target'] == True)), 'DTI'] = "DT"
# if target is not a therapeutic target, 'cpd_target_pair' cannot be in DTIs_set
# (~df_combined['cpd_target_pair'].isin(DTIs_set)) is included for clarity
df_combined.loc[((~df_combined['cpd_target_pair'].isin(DTIs_set)) 
                 & (df_combined['therapeutic_target'] == False)), 'DTI'] = "NDT"

In [26]:
############### TESTING: before discarding NDT rows ###############
add_dataset_sizes(df_combined, "pre DTI")

Discard rows that were annotated with NDT, i.e., compound-target pairs that are not in the drug_mechanisms table and for which the target was also not in the drug_mechanisms table (not a comparator compound).

In [27]:
# discard NDT rows
df_combined = df_combined[(df_combined['DTI'].isin(['D_DT', 'C3_DT', 'C2_DT', 'C1_DT', 'C0_DT', 'DT']))]

In [28]:
############### TESTING: after discarding NDT rows ###############
add_dataset_sizes(df_combined, "post DTI")

# Add Compound Properties Based on ChEMBL Data

## Add First Appearance of Compound in the Literature

Query and calculate the first appearance of a compound in the literature based on ChEMBL data.

In [29]:
# first appearance of a compound in the literature 
# information about salts is aggregated in the parent
sql = '''
SELECT DISTINCT docs.year, mh.parent_molregno
FROM docs
LEFT JOIN compound_records cr
    ON docs.doc_id = cr.doc_id
INNER JOIN molecule_hierarchy mh 
    ON cr.molregno = mh.molregno   -- cr.molregno = salt_molregno
WHERE docs.year is not null
'''

df_docs = pd.read_sql_query(sql, con=chembl_con)
df_docs['first_publication_cpd'] = df_docs.groupby('parent_molregno')['year'].transform('min')
df_docs = df_docs[['parent_molregno', 'first_publication_cpd']].drop_duplicates()
df_docs

,parent_molregno,first_publication_cpd
0,4941,1974
1,921,1974
2,1005421,1976
3,1750777,1976
4,1750778,1976
...,...,...
2024954,2737198,2022
2024956,2731421,2022
2024957,2728877,2022
2024958,2726635,2022


Combine with previous data.

In [30]:
df_combined = df_combined.merge(df_docs, on = 'parent_molregno', how='left')
df_combined = df_combined.astype({'first_publication_cpd': 'Int64'})
df_combined

,parent_molregno,tid_mutation,pchembl_value_mean_BF,pchembl_value_max_BF,pchembl_value_median_BF,first_publication_cpd_target_pair_BF,first_publication_cpd_target_pair_w_pchembl_BF,pchembl_value_mean_B,pchembl_value_max_B,pchembl_value_median_B,first_publication_cpd_target_pair_B,first_publication_cpd_target_pair_w_pchembl_B,parent_chemblid,parent_pref_name,max_phase,first_approval,usan_year,black_box_warning,prodrug,oral,parenteral,topical,tid,mutation,target_chembl_id,target_pref_name,target_type,organism,cpd_target_pair,keep_for_binding,keep_for_functional_binding,therapeutic_target,DTI,first_publication_cpd
0,100708,50,7.53,7.53,7.53,1982,1982,7.53,7.53,7.53,1982,1982,CHEMBL305153,None,NaN,<NA>,<NA>,0,-1,0,0,0,50,None,CHEMBL213,Beta-1 adrenergic receptor,SINGLE PROTEIN,Homo sapiens,100708_50,True,True,True,DT,1982
1,1798744,136,6.96,6.96,6.96,1986,1986,6.96,6.96,6.96,1986,1986,CHEMBL3350133,None,NaN,<NA>,<NA>,0,-1,0,0,0,136,None,CHEMBL236,Delta opioid receptor,SINGLE PROTEIN,Homo sapiens,1798744_136,True,True,True,DT,1986
2,88622,248,6.00,6.00,6.00,1987,1987,6.00,6.00,6.00,1987,1987,CHEMBL57825,None,NaN,<NA>,<NA>,0,-1,0,0,0,248,None,CHEMBL1835,Thromboxane-A synthase,SINGLE PROTEIN,Homo sapiens,88622_248,True,True,True,DT,1987
3,97517,72,5.01,5.01,5.01,1998,1998,5.01,5.01,5.01,1998,1998,CHEMBL303519,None,NaN,<NA>,<NA>,0,-1,0,0,0,72,None,CHEMBL217,Dopamine D2 receptor,SINGLE PROTEIN,Homo sapiens,97517_72,True,True,True,DT,1998
4,97517,130,5.25,5.25,5.25,1998,1998,5.25,5.25,5.25,1998,1998,CHEMBL303519,None,NaN,<NA>,<NA>,0,-1,0,0,0,130,None,CHEMBL234,Dopamine D3 receptor,SINGLE PROTEIN,Homo sapiens,97517_130,True,True,True,DT,1998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1062993,581767,11400,NaN,NaN,NaN,<NA>,<NA>,NaN,NaN,NaN,<NA>,<NA>,CHEMBL571209,INDOXIMOD,2.0,<NA>,2013.0,0,0,0,0,0,11400,NaN,CHEMBL2842,Serine/threonine-protein kinase mTOR,SINGLE PROTEIN,Homo sapiens,581767_11400,True,True,True,C2_DT,2010
1062994,1927154,201,NaN,NaN,NaN,<NA>,<NA>,NaN,NaN,NaN,<NA>,<NA>,CHEMBL3545305,CIBINETIDE,2.0,<NA>,2015.0,0,0,0,0,0,201,NaN,CHEMBL1817,Erythropoietin receptor,SINGLE PROTEIN,Homo sapiens,1927154_201,True,True,True,C2_DT,<NA>
1062995,16973,103450,NaN,NaN,NaN,<NA>,<NA>,NaN,NaN,NaN,<NA>,<NA>,CHEMBL277535,BIFONAZOLE,4.0,<NA>,1981.0,0,0,0,0,0,103450,NaN,CHEMBL1163121,3-hydroxy-3-methylglutaryl-coenzyme A reductase,SINGLE PROTEIN,Schizosaccharomyces pombe 972h-,16973_103450,True,True,True,D_DT,1995
1062996,1407411,148,NaN,NaN,NaN,<NA>,<NA>,NaN,NaN,NaN,<NA>,<NA>,CHEMBL2135460,TERLIPRESSIN,4.0,2022.0,2006.0,1,0,0,1,0,148,NaN,CHEMBL1921,Vasopressin V1b receptor,SINGLE PROTEIN,Homo sapiens,1407411_148,True,True,True,D_DT,2010


## Add ChEMBL Compound Properties and Compound Structures

Add compound properties and structures based on the compound_properties table and the compound_structures table. 

In [31]:
sql = '''
SELECT DISTINCT mh.parent_molregno, 
    cp.mw_freebase, cp.alogp, cp.hba, cp.hbd, cp.psa, cp.rtb, cp.ro3_pass, cp.num_ro5_violations, 
    cp.cx_most_apka, cp.cx_most_bpka, cp.cx_logp, cp.cx_logd, cp.molecular_species, cp.full_mwt, 
    cp.aromatic_rings, cp.heavy_atoms, cp.qed_weighted, cp.mw_monoisotopic, cp.full_molformula, 
    cp.hba_lipinski, cp.hbd_lipinski, cp.num_lipinski_ro5_violations, 
    struct.standard_inchi, struct.standard_inchi_key, struct.canonical_smiles
FROM compound_properties cp
INNER JOIN molecule_hierarchy mh
    ON cp.molregno = mh.parent_molregno
INNER JOIN compound_structures struct
    ON mh.parent_molregno = struct.molregno
'''

df_cpd_props = pd.read_sql_query(sql, con=chembl_con)
df_cpd_props.head()

,parent_molregno,mw_freebase,alogp,hba,hbd,psa,rtb,ro3_pass,num_ro5_violations,cx_most_apka,cx_most_bpka,cx_logp,cx_logd,molecular_species,full_mwt,aromatic_rings,heavy_atoms,qed_weighted,mw_monoisotopic,full_molformula,hba_lipinski,hbd_lipinski,num_lipinski_ro5_violations,standard_inchi,standard_inchi_key,canonical_smiles
0,2657369,411.94,3.82,4.0,2.0,64.26,5.0,N,0.0,11.94,6.11,3.98,3.96,NEUTRAL,411.94,3.0,29.0,0.67,411.1826,C22H26ClN5O,6.0,2.0,0.0,InChI=1S/C22H26ClN5O/c1-27(2)18-6-3-15(4-7-18)14-24-21(29)16-9-11-28(12-10-16)22-25-19-8-5-17(23...,AAAADVYFXUUVEO-UHFFFAOYSA-N,CN(C)c1ccc(CNC(=O)C2CCN(c3nc4ccc(Cl)cc4[nH]3)CC2)cc1
1,477782,506.37,3.04,8.0,2.0,116.43,8.0,N,1.0,NaN,6.50,2.16,2.11,NEUTRAL,506.37,2.0,27.0,0.53,506.0485,C17H23IN4O4S,8.0,3.0,1.0,"InChI=1S/C17H23IN4O4S/c1-10(2)11-7-14(25-3)12(18)8-13(11)26-15-9-21-17(22-16(15)19)20-5-6-27(4,2...",AAAAEENPAALFRN-UHFFFAOYSA-N,COc1cc(C(C)C)c(Oc2cnc(NCCS(C)(=O)=O)nc2N)cc1I
2,2237474,927.28,7.03,11.0,7.0,252.91,41.0,N,4.0,4.13,NaN,8.43,5.36,ACID,927.28,0.0,65.0,0.02,926.6555,C49H90N4O12,16.0,8.0,4.0,InChI=1S/C49H90N4O12/c1-5-8-10-12-14-16-18-20-22-24-26-28-30-37(31-29-27-25-23-21-19-17-15-13-11...,AAAAJHGLNDAXFP-VNKVACROSA-N,CCCCCCCCCCCCCCC(CCCCCCCCCCCCCC)C(=O)OC[C@H]1OC(O)[C@H](NC(C)=O)[C@@H](OCC(=O)N[C@@H](CC)C(=O)N[C...
3,412019,271.32,1.72,2.0,2.0,65.20,1.0,N,0.0,13.43,NaN,0.77,0.77,NEUTRAL,271.32,2.0,20.0,0.83,271.1321,C15H17N3O2,5.0,2.0,0.0,"InChI=1S/C15H17N3O2/c1-8-7-16-14(19)13-12(8)10-6-9(15(20)18(2)3)4-5-11(10)17-13/h4-6,8,17H,7H2,1...",AAAAKTROWFNLEP-UHFFFAOYSA-N,CC1CNC(=O)c2[nH]c3ccc(C(=O)N(C)C)cc3c21
4,26284,323.35,2.13,4.0,1.0,71.53,3.0,N,0.0,NaN,4.73,1.13,1.13,NEUTRAL,323.35,2.0,24.0,0.94,323.1270,C18H17N3O3,6.0,1.0,0.0,InChI=1S/C18H17N3O3/c1-11(22)20-10-17-16-8-14-7-12(13-3-2-6-19-9-13)4-5-15(14)21(16)18(23)24-17/...,AAAATQFUBIBQIS-IRXDYDNUSA-N,CC(=O)NC[C@@H]1OC(=O)N2c3ccc(-c4cccnc4)cc3C[C@@H]12


Combine with previous data.

In [32]:
df_combined = df_combined.merge(df_cpd_props, on = 'parent_molregno', how = 'inner')

In [33]:
############### TESTING: compound props ###############
add_dataset_sizes(df_combined, "cpd props")

## Calculate Ligand Efficiency (LE) Metrics

Calculate the ligand efficiency metrics for the compounds based on the mean pchembl values for a compound-target pair and the following ligand efficiency (LE) formulas:

$\text{LE} = \frac{\Delta\text{G}}{\text{HA}}$
where $ \Delta\text{G} = − RT \ln(K_d)$, $− RT\ln(K_i)$, or $− RT\ln(IC_{50})$

$\text{LE}=\frac{(2.303 \cdot 298 \cdot 0.00199 \cdot \text{pchembl_value})} {\text{heavy_atoms}}$


$\text{BEI}=\frac{\text{pchembl_mean} \cdot 1000} {\text{mw_freebase}}$

$\text{SEI}=\frac{\text{pchembl_mean} \cdot 100} {\text{PSA}}$

$\text{LLE}=\text{pchembl_mean}-\text{ALOGP}$

In [34]:
for assay_desc in ['BF', 'B']:
    df_combined['LE_'+assay_desc] = df_combined['pchembl_value_mean_'+assay_desc]/df_combined['heavy_atoms']*(2.303*298*0.00199)
    df_combined['BEI_'+assay_desc] = df_combined['pchembl_value_mean_'+assay_desc]*1000/df_combined["mw_freebase"]
    df_combined['SEI_'+assay_desc] = df_combined['pchembl_value_mean_'+assay_desc]*100/df_combined["psa"]
    df_combined['LLE_'+assay_desc] = df_combined['pchembl_value_mean_'+assay_desc]-df_combined["alogp"]

## Add ATC Classifications (Level 1)

Query ATC classifications (level 1) from the atc_classification and molecule_atc_classification tables.

In [35]:
sql = '''
SELECT DISTINCT mh.parent_molregno, atc.level1, level1_description
FROM atc_classification atc
INNER JOIN molecule_atc_classification matc
    ON atc.level5 = matc.level5
INNER JOIN molecule_hierarchy mh
    ON matc.molregno = mh.molregno
'''

atc_levels = pd.read_sql_query(sql, con=chembl_con)
atc_levels["l1_full"] = atc_levels["level1"] + "_" + atc_levels["level1_description"]
atc_levels

,parent_molregno,level1,level1_description,l1_full
0,628156,L,ANTINEOPLASTIC AND IMMUNOMODULATING AGENTS,L_ANTINEOPLASTIC AND IMMUNOMODULATING AGENTS
1,2286380,L,ANTINEOPLASTIC AND IMMUNOMODULATING AGENTS,L_ANTINEOPLASTIC AND IMMUNOMODULATING AGENTS
2,2089491,L,ANTINEOPLASTIC AND IMMUNOMODULATING AGENTS,L_ANTINEOPLASTIC AND IMMUNOMODULATING AGENTS
3,608601,L,ANTINEOPLASTIC AND IMMUNOMODULATING AGENTS,L_ANTINEOPLASTIC AND IMMUNOMODULATING AGENTS
4,1927225,L,ANTINEOPLASTIC AND IMMUNOMODULATING AGENTS,L_ANTINEOPLASTIC AND IMMUNOMODULATING AGENTS
...,...,...,...,...
4003,675276,A,ALIMENTARY TRACT AND METABOLISM,A_ALIMENTARY TRACT AND METABOLISM
4004,2197623,N,NERVOUS SYSTEM,N_NERVOUS SYSTEM
4005,1383224,C,CARDIOVASCULAR SYSTEM,C_CARDIOVASCULAR SYSTEM
4006,675183,A,ALIMENTARY TRACT AND METABOLISM,A_ALIMENTARY TRACT AND METABOLISM


Combine ATC level annotations for the same parent_molregno into one description.

In [36]:
between_str_join = ' | '
atc_levels['atc_level1'] = atc_levels.groupby(['parent_molregno'])['l1_full'].transform(lambda x: between_str_join.join(sorted(x)))
atc_levels = atc_levels[['parent_molregno', 'atc_level1']].drop_duplicates()
atc_levels

,parent_molregno,atc_level1
0,628156,L_ANTINEOPLASTIC AND IMMUNOMODULATING AGENTS
1,2286380,L_ANTINEOPLASTIC AND IMMUNOMODULATING AGENTS
2,2089491,L_ANTINEOPLASTIC AND IMMUNOMODULATING AGENTS
3,608601,L_ANTINEOPLASTIC AND IMMUNOMODULATING AGENTS
4,1927225,L_ANTINEOPLASTIC AND IMMUNOMODULATING AGENTS
...,...,...
4003,675276,A_ALIMENTARY TRACT AND METABOLISM
4004,2197623,N_NERVOUS SYSTEM
4005,1383224,C_CARDIOVASCULAR SYSTEM
4006,675183,A_ALIMENTARY TRACT AND METABOLISM


Combine with previous data.

In [37]:
df_combined = df_combined.merge(atc_levels, on='parent_molregno', how = 'left')

# Add Target Class Annotations Based on ChEMBL Data

Add information about level 1 and level 2 target class annotations in ChEMBL.

In [38]:
sql = '''
SELECT DISTINCT tc.tid, 
    pc.protein_class_id, pc.pref_name, pc.short_name, pc.protein_class_desc, pc.definition
FROM protein_classification pc
-- join several tables to get the corresponding target id
INNER JOIN component_class cc
    ON pc.protein_class_id = cc.protein_class_id
INNER JOIN component_sequences cs
    ON cc.component_id = cs.component_id
INNER JOIN target_components tc
    ON cs.component_id = tc.component_id
'''

df_target_classes = pd.read_sql_query(sql, con=chembl_con)
# only interested in the target ids that are in the current dataset
current_tids = set(df_combined['tid'])
df_target_classes = df_target_classes[df_target_classes['tid'].isin(current_tids)]
df_target_classes

,tid,protein_class_id,pref_name,short_name,protein_class_desc,definition
0,1,646,Hydrolase,Hydrolase,enzyme hydrolase,A group of enzymes that catalyze the hydrolysis of a chemical bond
1,2,1133,ABCC subfamily,MRP,transporter ntpase atp binding cassette mrp,A sequence-related subfamily of ATP-BINDING CASSETTE TRANSPORTERS that actively transport organi...
2,3,104,Phosphodiesterase 5A,PDE_5A,enzyme phosphodiesterase pde_5 pde_5a,None
3,4,1583,Voltage-gated calcium channel,VG CA,ion channel vgc vg ca,Voltage-dependent cell membrane glycoproteins selectively permeable to calcium ions. They are ca...
4,5,422,Nicotinic acetylcholine receptor alpha subunit,CHRN alpha,ion channel lgic ach chrn alpha,None
...,...,...,...,...,...,...
11492,120332,11,Membrane receptor,Membrane receptor,membrane receptor,Cell surface proteins that bind signalling molecules external to the cell with high affinity and...
11501,120341,10,Oxidoreductase,Reductase,enzyme reductase,The class of all enzymes catalyzing oxidoreduction reactions. The substrate that is oxidized is ...
11530,120396,601,Unclassified protein,Unclassified,unclassified,None
11532,120399,89,Threonine protease T1A subfamily,T1A,enzyme protease threonine pbt t1a,None


Query the protein_classification table for the protein classification hierarchy and merge it with the target class information for specific tids.

In [39]:
sql = '''
WITH RECURSIVE pc_hierarchy AS (
    SELECT protein_class_id,
            parent_id,
            class_level,
            pref_name AS names
    FROM protein_classification
    WHERE parent_id IS NULL

    UNION ALL
   
    SELECT pc.protein_class_id,
        pc.parent_id,
        pc.class_level,
        -- recursively add current protein classification pref_name to string, separated by |
        pc_hierarchy.names || '|' || pc.pref_name 
    FROM protein_classification pc, pc_hierarchy
    WHERE pc.parent_id = pc_hierarchy.protein_class_id
)
SELECT *
FROM pc_hierarchy
'''


target_class_hierarchy = pd.read_sql_query(sql, con=chembl_con)
target_class_hierarchy[['l0', 'l1', 'l2', 'l3', 'l4', 'l5', 'l6']] = target_class_hierarchy['names'].str.split('|', expand=True)
target_class_hierarchy = target_class_hierarchy[target_class_hierarchy['protein_class_id'] != 0][['protein_class_id', 'l1', 'l2']].sort_values(by=['protein_class_id']).reset_index(drop=True)
df_target_classes = df_target_classes.merge(target_class_hierarchy, on='protein_class_id', how = 'left')
df_target_classes

,tid,protein_class_id,pref_name,short_name,protein_class_desc,definition,l1,l2
0,1,646,Hydrolase,Hydrolase,enzyme hydrolase,A group of enzymes that catalyze the hydrolysis of a chemical bond,Enzyme,Hydrolase
1,2,1133,ABCC subfamily,MRP,transporter ntpase atp binding cassette mrp,A sequence-related subfamily of ATP-BINDING CASSETTE TRANSPORTERS that actively transport organi...,Transporter,Primary active transporter
2,3,104,Phosphodiesterase 5A,PDE_5A,enzyme phosphodiesterase pde_5 pde_5a,None,Enzyme,Phosphodiesterase
3,4,1583,Voltage-gated calcium channel,VG CA,ion channel vgc vg ca,Voltage-dependent cell membrane glycoproteins selectively permeable to calcium ions. They are ca...,Ion channel,Voltage-gated ion channel
4,5,422,Nicotinic acetylcholine receptor alpha subunit,CHRN alpha,ion channel lgic ach chrn alpha,None,Ion channel,Ligand-gated ion channel
...,...,...,...,...,...,...,...,...
1543,120332,11,Membrane receptor,Membrane receptor,membrane receptor,Cell surface proteins that bind signalling molecules external to the cell with high affinity and...,Membrane receptor,None
1544,120341,10,Oxidoreductase,Reductase,enzyme reductase,The class of all enzymes catalyzing oxidoreduction reactions. The substrate that is oxidized is ...,Enzyme,Oxidoreductase
1545,120396,601,Unclassified protein,Unclassified,unclassified,None,Unclassified protein,None
1546,120399,89,Threonine protease T1A subfamily,T1A,enzyme protease threonine pbt t1a,None,Enzyme,Protease


Summarise the information for a target id with several assigned target classes of level 1 into one description. If a target id has more than one assigned target class, the target class 'Unclassified protein' is discarded.

In [40]:
level = 'l1'
between_str_join = '|'
target_classes_level = df_target_classes[['tid', level]].drop_duplicates().dropna()

# remove 'Unclassified protein' from targets with more than one target class, level 1
more_than_one = target_classes_level.groupby(['tid'])[level].count()
target_classes_level = target_classes_level[
    (target_classes_level['tid'].isin(more_than_one[more_than_one == 1].index.tolist())) 
    | ((target_classes_level['tid'].isin(more_than_one[more_than_one > 1].index.tolist())) 
       & (target_classes_level['l1'] != 'Unclassified protein'))]

target_classes_level['target_class_l1'] = target_classes_level.groupby(['tid'])[level].transform(lambda x: between_str_join.join(sorted(x)))
target_classes_level = target_classes_level[['tid', 'target_class_l1']].drop_duplicates()

df_combined = df_combined.merge(target_classes_level, on='tid', how = 'left')

Repeat the summary step for target classes of level 2.

In [41]:
level = 'l2'
target_classes_level = df_target_classes[['tid', level]].drop_duplicates().dropna()
target_classes_level['target_class_l2'] = target_classes_level.groupby(['tid'])[level].transform(lambda x: between_str_join.join(sorted(x)))
target_classes_level = target_classes_level[['tid', 'target_class_l2']].drop_duplicates()

df_combined = df_combined.merge(target_classes_level, on='tid', how = 'left')

Instances with targets with more than one target class assigned to them.  
These could be reassigned by hand if a single target class is preferable.

In [42]:
############### TESTING: which targets have more than one level 1 target class assigned to them? ###############
test = df_combined[(~df_combined['target_class_l1'].isnull()) & (df_combined['target_class_l1'].str.contains('|', regex=False))][['tid', 'target_pref_name', 'target_type', 'target_class_l1', 'target_class_l2']].drop_duplicates()
print("#Instances with >1 level 1 target class:", len(test))
test

#Instances with >1 level 1 target class: 41


,tid,target_pref_name,target_type,target_class_l1,target_class_l2
22,11531,Nuclear factor NF-kappa-B p105 subunit,SINGLE PROTEIN,Other cytosolic protein|Transcription factor,NaN
400,104295,Cyclin-dependent kinase 4/cyclin D1,PROTEIN COMPLEX,Enzyme|Other cytosolic protein,Kinase
2130,105036,Atrial natriuretic peptide receptor,PROTEIN FAMILY,Enzyme|Membrane receptor,Lyase
3727,104811,Bcr/Abl fusion protein,CHIMERIC PROTEIN,Enzyme|Other cytosolic protein,Kinase
4992,109099,Platelet glycoprotein VI (GPVI),SINGLE PROTEIN,Adhesion|Membrane receptor,NaN
5615,105079,NPM/ALK (Nucleophosmin/ALK tyrosine kinase receptor),CHIMERIC PROTEIN,Enzyme|Other nuclear protein,Kinase
5725,100128,Breakpoint cluster region protein,SINGLE PROTEIN,Enzyme|Other cytosolic protein,Kinase
6263,104841,Serotonin (5-HT) receptor,PROTEIN FAMILY,Ion channel|Membrane receptor,Family A G protein-coupled receptor|Ligand-gated ion channel
6652,104737,Sulfonylurea receptors; K-ATP channels,PROTEIN COMPLEX GROUP,Ion channel|Transporter,Primary active transporter|Voltage-gated ion channel
7605,104730,Nuclear factor NF-kappa-B complex,PROTEIN COMPLEX GROUP,Other cytosolic protein|Transcription factor,NaN


In [43]:
############### TESTING: which targets have more than one level 2 target class assigned to them? ###############df_combined_test = df_combined[~(df_combined['target_class_l2'].isnull())]
test = df_combined[(~df_combined['target_class_l2'].isnull()) & (df_combined['target_class_l2'].str.contains('|', regex=False))][['tid', 'target_pref_name', 'target_type', 'target_class_l1', 'target_class_l2']].drop_duplicates()
print("#Instances with >1 level 2 target class:", len(test))
test

#Instances with >1 level 2 target class: 22


,tid,target_pref_name,target_type,target_class_l1,target_class_l2
21,104677,Menin/Histone-lysine N-methyltransferase MLL,PROTEIN-PROTEIN INTERACTION,Epigenetic regulator,Reader|Writer
23,103732,Histone-lysine N-methyltransferase MLL,SINGLE PROTEIN,Epigenetic regulator,Reader|Writer
4351,101310,CREB-binding protein,SINGLE PROTEIN,Epigenetic regulator,Reader|Writer
6263,104841,Serotonin (5-HT) receptor,PROTEIN FAMILY,Ion channel|Membrane receptor,Family A G protein-coupled receptor|Ligand-gated ion channel
6652,104737,Sulfonylurea receptors; K-ATP channels,PROTEIN COMPLEX GROUP,Ion channel|Transporter,Primary active transporter|Voltage-gated ion channel
7478,106175,Cytochrome P450 3A,PROTEIN FAMILY,Enzyme,Cytochrome P450|Oxidoreductase
8478,104758,Potassium-transporting ATPase,PROTEIN COMPLEX,Enzyme|Transporter,Hydrolase|Primary active transporter
8728,104782,"Sulfonylurea receptor 2, Kir6.2",PROTEIN COMPLEX,Ion channel|Transporter,Primary active transporter|Voltage-gated ion channel
10075,104770,Sodium/potassium-transporting ATPase,PROTEIN COMPLEX GROUP,Enzyme|Ion channel|Transporter,Hydrolase|Other ion channel|Primary active transporter
10465,29,Sodium/potassium-transporting ATPase alpha-1 chain,SINGLE PROTEIN,Enzyme|Transporter,Hydrolase|Primary active transporter


# Add RDKit-Based Compound Descriptors

## Built-in Compound Descriptors

Add relevant compound descriptors using built-in RDKit methods. 

In [44]:
# # add a column with RDKit molecules, used to calculate the descriptors
# PandasTools.AddMoleculeColumnToFrame(df_combined,'canonical_smiles','mol',includeFingerprints=False)

# df_combined.loc[:,'fraction_csp3'] = df_combined['mol'].apply(Descriptors.FractionCSP3)
# df_combined.loc[:,'num_aliphatic_carbocycles'] = df_combined['mol'].apply(Descriptors.NumAliphaticCarbocycles)
# df_combined.loc[:,'num_aliphatic_heterocycles'] = df_combined['mol'].apply(Descriptors.NumAliphaticHeterocycles)
# df_combined.loc[:,'num_aliphatic_rings'] = df_combined['mol'].apply(Descriptors.NumAliphaticRings)
# df_combined.loc[:,'num_aromatic_carbocycles'] = df_combined['mol'].apply(Descriptors.NumAromaticCarbocycles)
# df_combined.loc[:,'num_aromatic_heterocycles'] = df_combined['mol'].apply(Descriptors.NumAromaticHeterocycles)
# df_combined.loc[:,'num_aromatic_rings'] = df_combined['mol'].apply(Descriptors.NumAromaticRings)
# df_combined.loc[:,'num_heteroatoms'] = df_combined['mol'].apply(Descriptors.NumHeteroatoms)
# df_combined.loc[:,'num_saturated_carbocycles'] = df_combined['mol'].apply(Descriptors.NumSaturatedCarbocycles)
# df_combined.loc[:,'num_saturated_heterocycles'] = df_combined['mol'].apply(Descriptors.NumSaturatedHeterocycles)
# df_combined.loc[:,'num_saturated_rings'] = df_combined['mol'].apply(Descriptors.NumSaturatedRings)
# df_combined.loc[:,'ring_count'] = df_combined['mol'].apply(Descriptors.RingCount)
# df_combined.loc[:,'num_stereocentres'] = df_combined['mol'].apply(Chem.rdMolDescriptors.CalcNumAtomStereoCenters)

# # drop the column with RDKit molecules
# df_combined = df_combined.drop(['mol'] , axis=1)

## Aromaticity Descriptors

Add descriptors for aromaticity, using an RDKit-based method.

In [45]:
def calculate_aromatic_atoms(smiles_set):
    aromatic_atoms_dict = dict()
    aromatic_c_dict = dict()
    aromatic_n_dict = dict()
    aromatic_hetero_dict = dict()
    
    for smiles in tqdm(smiles_set):
        mol = Chem.MolFromSmiles(smiles)
        aromatic_atoms_dict[smiles] = sum(mol.GetAtomWithIdx(i).GetIsAromatic() for i in range(mol.GetNumAtoms()))
        aromatic_c_dict[smiles] = sum((mol.GetAtomWithIdx(i).GetIsAromatic() & (mol.GetAtomWithIdx(i).GetAtomicNum() == 6)) for i in range(mol.GetNumAtoms()))
        aromatic_n_dict[smiles] = sum((mol.GetAtomWithIdx(i).GetIsAromatic() & (mol.GetAtomWithIdx(i).GetAtomicNum() == 7)) for i in range(mol.GetNumAtoms()))
        aromatic_hetero_dict[smiles] = sum((mol.GetAtomWithIdx(i).GetIsAromatic() & (mol.GetAtomWithIdx(i).GetAtomicNum() != 6) & (mol.GetAtomWithIdx(i).GetAtomicNum() != 1)) for i in range(mol.GetNumAtoms()))
        
    return aromatic_atoms_dict, aromatic_c_dict, aromatic_n_dict, aromatic_hetero_dict

In [46]:
# smiles_set = set(df_combined["canonical_smiles"])
# aromatic_atoms_dict, aromatic_c_dict, aromatic_n_dict, aromatic_hetero_dict = calculate_aromatic_atoms(list(smiles_set))

# df_combined['aromatic_atoms'] = df_combined['canonical_smiles'].map(aromatic_atoms_dict)
# df_combined['aromatic_c'] = df_combined['canonical_smiles'].map(aromatic_c_dict)
# df_combined['aromatic_n'] = df_combined['canonical_smiles'].map(aromatic_n_dict)
# df_combined['aromatic_hetero'] = df_combined['canonical_smiles'].map(aromatic_hetero_dict)

## Scaffold SMILES

Add the scaffold SMILES for every molecule. For the column *scaffold_w_stereo* the stereochemistry is taken into account. For the column *scaffold_wo_stereo* the stereochemistry information is removed before calculating the scaffold.

In [47]:
# note: this takes a few minutes to calculate for all molecules
def calculate_scaffolds(smiles_set):
    scaffolds_dict = dict()
    scaffolds_no_stereo_dict = dict()
    
    for smiles in tqdm(smiles_set):
        mol = Chem.MolFromSmiles(smiles)
        # skip acyclic molecules
        if Chem.rdMolDescriptors.CalcNumRings(mol) == 0:
            continue

        scaffold = MurckoScaffold.GetScaffoldForMol(mol)
        scaffolds_dict[smiles] = Chem.MolToSmiles(scaffold)
        
        # repeat after removing stereochemistry
        Chem.RemoveStereochemistry(mol)
        scaffold_no_stereo = MurckoScaffold.GetScaffoldForMol(mol)
        scaffolds_no_stereo_dict[smiles] = Chem.MolToSmiles(scaffold_no_stereo)
        
    return scaffolds_dict, scaffolds_no_stereo_dict

In [48]:
# smiles_set = set(df_combined["canonical_smiles"])
# scaffolds_dict, scaffolds_no_stereo_dict = calculate_scaffolds(smiles_set)

# df_combined["scaffold_w_stereo"] = df_combined['canonical_smiles'].map(scaffolds_dict)
# df_combined['scaffold_wo_stereo'] = df_combined['canonical_smiles'].map(scaffolds_no_stereo_dict)

# Get Relevant Subsets of the Data

Calculate different subsets of the data based on binding and functional data in ChEMBL.

In [49]:
# function to calculate and return the different subsets of interest
def get_data_subsets(min_nof_cpds, data, desc):
    if desc == 'B':
        drop_desc = 'BF'
    else:
        drop_desc = 'B'
    data = data.drop(columns=['pchembl_value_mean_'+drop_desc, 
                              'pchembl_value_max_'+drop_desc, 
                              'pchembl_value_median_'+drop_desc, 
                              'first_publication_cpd_target_pair_'+drop_desc, 
                              'first_publication_cpd_target_pair_w_pchembl_'+drop_desc, 
                              'LE_'+drop_desc,
                              'BEI_'+drop_desc,
                              'SEI_'+drop_desc,
                              'LLE_'+drop_desc]).drop_duplicates()
    
    # Restrict the dataset to targets with at least *min_nof_cpds* compounds with a pchembl value.
    comparator_counts = data[~data['pchembl_value_mean_'+desc].isnull()].groupby(['tid_mutation'])['parent_molregno'].count()
    targets_w_enough_cpds = comparator_counts[comparator_counts >= min_nof_cpds].index.tolist()
    df_enough_cpds = data.query('tid_mutation in @targets_w_enough_cpds')
    
    # Restrict the dataset further to targets with at least one compound-target pair labelled as 'D_DT', 'C3_DT', 'C2_DT', 'C1_DT' or 'C0_DT', 
    # i.e. a compound-target pair with a known interaction.
    c_dt_d_dt_targets = set(df_enough_cpds[df_enough_cpds['DTI'].isin(['D_DT', 'C3_DT', 'C2_DT', 'C1_DT', 'C0_DT'])].tid_mutation.to_list())
    df_c_dt_d_dt = df_enough_cpds.query('tid_mutation in @c_dt_d_dt_targets')
    
    # Restrict the dataset further to targets with at least one compound-target pair labelled as 'D_DT', 
    # i.e. a known drug-target interaction. 
    d_dt_targets = set(df_enough_cpds[df_enough_cpds['DTI'] == 'D_DT'].tid_mutation.to_list())
    df_d_dt = df_enough_cpds.query('tid_mutation in @d_dt_targets')
    
    return df_enough_cpds, df_c_dt_d_dt, df_d_dt

## Binding and Functional Assays

In [50]:
# consider binding and functional assays
min_nof_cpds = 100
# assay description = binding+functional
desc = 'BF'
df_combined_all = df_combined[df_combined['keep_for_functional_binding'] == True].copy()
df_combined_all_enough_cpds, df_combined_all_c_dt_d_dt, df_combined_all_d_dt = get_data_subsets(min_nof_cpds, df_combined_all, desc)

In [51]:
# df_combined_all.to_csv(path_results+"ChEMBL"+chembl_version+"_DTI_all_assays.csv", sep = ";", index = False)
# # df_combined_all.to_excel(path_results+"ChEMBL"+chembl_version+"_DTI_all_assays.xlsx", index = False)

# df_combined_all_enough_cpds.to_csv(path_results+"ChEMBL"+chembl_version+"_DTI_all_assays_" + str(min_nof_cpds) + ".csv", sep = ";", index = False)
# # df_combined_all_enough_cpds.to_excel(path_results+"ChEMBL"+chembl_version+"_DTI_all_assays_" + str(min_nof_cpds) + ".xlsx", index = False)

# df_combined_all_c_dt_d_dt.to_csv(path_results+"ChEMBL"+chembl_version+"_DTI_all_assays_" + str(min_nof_cpds) + "_c_dt_d_dt.csv", sep = ";", index = False)
# # df_combined_all_c_dt_d_dt.to_excel(path_results+"ChEMBL"+chembl_version+"_DTI_all_assays_" + str(min_nof_cpds) + "_c_dt_d_dt.xlsx", index = False)

# df_combined_all_d_dt.to_csv(path_results+"ChEMBL"+chembl_version+"_DTI_all_assays_" + str(min_nof_cpds) + "_d_dt.csv", sep = ";", index = False)
# # df_combined_all_d_dt.to_excel(path_results+"ChEMBL"+chembl_version+"_DTI_all_assays_" + str(min_nof_cpds) + "_d_dt.xlsx", index = False)

In [52]:
############### TESTING: binding and functional assays ###############
add_dataset_sizes(df_combined_all, "all assays")
add_dataset_sizes(df_combined_all_enough_cpds, "all, >= 100")
add_dataset_sizes(df_combined_all_c_dt_d_dt, "all, >= 100, c_dt and d_dt")
add_dataset_sizes(df_combined_all_d_dt, "all, >= 100, d_dt")

## Only Binding Assays

In [53]:
# consider only binding assays
min_nof_cpds = 100
# assay description = binding
desc = 'B'
df_combined_B = df_combined[df_combined['keep_for_binding'] == True].copy()
df_combined_B_enough_cpds, df_combined_B_c_dt_d_dt, df_combined_B_d_dt = get_data_subsets(min_nof_cpds, df_combined_B, desc)

In [54]:
# df_combined_B.to_csv(path_results+"ChEMBL"+chembl_version+"_DTI_binding.csv", sep = ";", index = False)
# # df_combined_B.to_excel(path_results+"ChEMBL"+chembl_version+"_DTI_binding.xlsx", index = False)

# df_combined_B_enough_cpds.to_csv(path_results+"ChEMBL"+chembl_version+"_DTI_binding_" + str(min_nof_cpds) + ".csv", sep = ";", index = False)
# # df_combined_B_enough_cpds.to_excel(path_results+"ChEMBL"+chembl_version+"_DTI_binding_" + str(min_nof_cpds) + ".xlsx", index = False)

# df_combined_B_c_dt_d_dt.to_csv(path_results+"ChEMBL"+chembl_version+"_DTI_binding_" + str(min_nof_cpds) + "_c_dt_d_dt.csv", sep = ";", index = False)
# # df_combined_B_c_dt_d_dt.to_excel(path_results+"ChEMBL"+chembl_version+"_DTI_binding_" + str(min_nof_cpds) + "_c_dt_d_dt.xlsx", index = False)

# df_combined_B_d_dt.to_csv(path_results+"ChEMBL"+chembl_version+"_DTI_binding_" + str(min_nof_cpds) + "_d_dt.csv", sep = ";", index = False)
# # df_combined_B_d_dt.to_excel(path_results+"ChEMBL"+chembl_version+"_DTI_binding_" + str(min_nof_cpds) + "_d_dt.xlsx", index = False)

In [55]:
############### TESTING: binding assays ###############
add_dataset_sizes(df_combined_B, "binding")
add_dataset_sizes(df_combined_B_enough_cpds, "b, >= 100")
add_dataset_sizes(df_combined_B_c_dt_d_dt, "b, >= 100, c_dt and d_dt")
add_dataset_sizes(df_combined_B_d_dt, "b, >= 100, d_dt")

# Testing: Overview of Dataset Sizes at Different Points in the Pipeline

In [56]:
############### TESTING: development of the full dataset size ###############
print("Size of full dataset at different points.")
pd.DataFrame(all_lengths,
                   columns=['type', 
                            '#mols', '#drugs', 
                            '#targets', '#drug_ targets', 
                            '#targets_ mutation', '#drug_ targets_mutation', 
                            '#cpd_tid_ pairs', '#drug_tid_ pairs',
                            '#cpd_ tid_mutation_ pairs', '#drug_ tid_mutation_ pairs'])

Size of full dataset at different points.


,type,#mols,#drugs,#targets,#drug_ targets,#targets_ mutation,#drug_ targets_mutation,#cpd_tid_ pairs,#drug_tid_ pairs,#cpd_ tid_mutation_ pairs,#drug_ tid_mutation_ pairs
0,init,1020560,1760,6825,2585,8204,2989,2251022,23237,2264746,24318
1,pre dm table,1017472,1731,6759,2545,8123,2947,2224733,22080,2238003,23159
2,pre DTI,1019952,2158,7358,1031,8724,1246,2233315,5786,2246600,6304
3,post DTI,674519,2158,1739,1031,2644,1246,1051446,5786,1062860,6304
4,cpd props,672233,1764,1411,862,2316,1077,1047840,5174,1059216,5692
5,all assays,672233,1764,1411,862,2316,1077,1047840,5174,1059216,5692
6,"all, >= 100",664427,1361,698,415,729,440,1029985,2846,1035722,2954
7,"all, >= 100, c_dt and d_dt",597535,1361,651,415,679,440,890695,2846,896173,2954
8,"all, >= 100, d_dt",380529,1361,415,415,440,440,574515,2846,579718,2954
9,binding,518513,1764,1409,862,2227,1072,751941,5174,762870,5687


In [57]:
############### TESTING: development of the dataset size (pchembl values required) ###############
print("Size of dataset with any pchembl values at different points.")
print("This includes data for which we only have pchembl data for functional assays but not for binding assays.")
pd.DataFrame(all_lengths_pchembl,
                   columns=['type', 
                            '#mols', '#drugs', 
                            '#targets', '#drug_ targets', 
                            '#targets_ mutation', '#drug_ targets_mutation', 
                            '#cpd_tid_ pairs', '#drug_tid_ pairs',
                            '#cpd_ tid_mutation_ pairs', '#drug_ tid_mutation_ pairs'])

Size of dataset with any pchembl values at different points.
This includes data for which we only have pchembl data for functional assays but not for binding assays.


,type,#mols,#drugs,#targets,#drug_ targets,#targets_ mutation,#drug_ targets_mutation,#cpd_tid_ pairs,#drug_tid_ pairs,#cpd_ tid_mutation_ pairs,#drug_ tid_mutation_ pairs
0,init,1020560,1760,6825,2585,8204,2989,2251022,23237,2264746,24318
1,pre dm table,1017472,1731,6759,2545,8123,2947,2224733,22080,2238003,23159
2,pre DTI,1017472,832,6759,419,8123,633,2224733,1484,2238003,2000
3,post DTI,671855,832,1145,419,2048,633,1042983,1484,1054382,2000
4,cpd props,671115,830,1145,419,2048,633,1041815,1482,1053176,1998
5,all assays,671115,830,1145,419,2048,633,1041815,1482,1053176,1998
6,"all, >= 100",663583,791,698,336,729,360,1027228,1360,1032956,1466
7,"all, >= 100, c_dt and d_dt",596662,791,651,336,679,360,887938,1360,893407,1466
8,"all, >= 100, d_dt",379677,791,415,336,440,360,572033,1360,577227,1466
9,binding,517186,785,1134,410,1950,619,745779,1415,756693,1926
